# IDENTIFICATION OF NOVEL CLASSES OF NEOANTIGENS IN CANCER | Tumor specific genes selection

In [ ]:
%load_ext rpy2.ipython

## 0. Data preparation

This first cell should be modified according to the data that is going to be used. It is only available for datasets with paired samples per patient: normal and tumor. 

The **PROJECT** variable should be changed according to the GEO identifier.

From the GEO website, the *SRR_Acc_List.txt* and *SraRunTable.txt* files should be manually downloaded and save in a directory. This directory should be specified in **SRR** variable.

The pipeline is developed with the intention of running the most computationally expensive programs in a cluster. 
In this case, a Gluster File System has been used. The code to run on a cluster may need to be adapted.

In [ ]:
import os,re,shutil,glob,openpyxl
import pandas as pd
from Bio import SeqIO
from gtfparse import read_gtf
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib import pyplot as plt
from IPython.display import Image

PROJECT="GSE193567"

DIR=os.path.join("data",PROJECT)

try:
    os.makedirs(DIR) #path where to store all the itermediate steps and outputs of the pipeline
except:
    print("Directory for %s already exists" %PROJECT)
    
CLUSTERDIR="/users/genomics/marta" #path where to run and store things that run in a cluster
SRR="/projects_eg/datasets/"+PROJECT # path where SRR_Acc_List.txt and SraRunTable.txt are stored. It should be inside a folder named with GEO accession
SRR_ACC=os.path.join(SRR,"SRR_Acc_List.txt") 
SRA=os.path.join(SRR,"SraRunTable.txt")

FASTQDIR=os.path.join(DIR,"fastq_files") #path where to store fastq files
try:
    os.mkdir(FASTQDIR)
except:
    print("Fastq_files directory exists")
    
shutil.copy(SRR_ACC, os.path.join(FASTQDIR,"SRR_Acc_List.txt"))
shutil.copy(SRA, os.path.join(FASTQDIR,"SraRunTable.txt"))

GENOMEDIR="genomes"

try:
    os.makedirs(os.path.join(DIR,"analysis"))
    os.makedirs(os.path.join(DIR,"results"))
    #os.makedirs(os.path.join(DIR,"scripts"))
except:
    print("Directory exists")



In [ ]:
%%R

require(tidyr)
require(dplyr)
require(rtracklayer)
#library(purrr)
require(ggplot2)
require(RColorBrewer)
require(devtools)
require(stringr)
require(edgeR)

Get a three column file with patient_id normal_id tumor_id for latter usage 

In [ ]:
metadata = pd.read_csv(os.path.join(FASTQDIR.split("/fastq_files")[0],"SraRunTable.txt"))
metadata = metadata[['Run','Individual','tissue']]

normal = metadata[metadata['tissue'] == "non-tumor"]
normal = normal[['Individual','Run']]

tumor = metadata[metadata['tissue'] == "tumor"]
tumor = tumor[['Individual','Run']].rename(columns ={'Run' : 'Run_t'})

patients = pd.merge(normal, tumor, on=['Individual'])
patients['Individual'] = patients['Individual'].str.split(' ').str[1]
patients.to_csv(os.path.join(DIR,"results/patients.csv"),index=False, header=False)
patients_summary = os.path.join(DIR,"results/patients.csv")

patients_id=list(patients.iloc[:,0])
normal_id=list(patients.iloc[:,1])
tumor_id=list(patients.iloc[:,2])

patients

## 07.Quantification with featureCounts



In [ ]:
%%bash -s "$DIR"

mkdir $1/analysis/07_quantification

In [ ]:
%%bash -s "$PROJECT" "$CLUSTERDIR" "$DIR" "$patients_summary"

######################################DONE IN CLUSTER###############################################

sbatch $3/scripts/2_tumor_specific/loop_featureCounts.sh $1 $2 $3 $4 

Process featureCounts output

In [ ]:
%%bash -s "$DIR"

mkdir $1/results/plots

In [ ]:
%%R -i patients_summary,DIR,patients

############PATIENTS INFO#############
colnames(patients) <- c("patient", "normal", "tumor")
patients_full <- patients %>% pivot_longer(cols = !patient, names_to = "normal_tumor", values_to = "sample")

###To wrangle table of counts
for (i in 1:nrow(patients)) {
    #read and process table of counts
    fc_res <- read.table(paste0(DIR,"/analysis/07_quantification/",patients[i,1],"/",patients[i,1],"_featureCounts.txt"), header=T, fill=T) 

    table_of_counts <- fc_res[, -c(1:6)]
    row.names(table_of_counts) <- fc_res$Geneid
    sample_names <- str_sub(colnames(table_of_counts),-40,-30)
    colnames(table_of_counts) <- sample_names
    table_of_counts$gene_id <- fc_res$Geneid
    table_of_counts <- table_of_counts[, c(3, 1, 2)]
    names(table_of_counts)[1] <- "gene_id"
    write.csv(table_of_counts, file=paste0(DIR,"/analysis/07_quantification/",patients[i,1],"/",patients[i,1],"_table_of_counts.csv"), row.names = FALSE, quote = FALSE)


    featureLength <- dplyr::select(fc_res, Length, Geneid)

    ###FROM COUNTS TO FPKM
    y <- DGEList(counts=table_of_counts[,c(2,3)],genes=data.frame(Length=featureLength$Length))
    y <- calcNormFactors(y)
    RPKM_raw <- as.data.frame(rpkm(y)) 
    RPKM <- RPKM_raw
    RPKM$gene_id <- rownames(RPKM)
    RPKM <- RPKM[, c(3, 1, 2)]
    novel_RPKM <- RPKM[grepl("STRG", RPKM$gene_id),] 
    known_RPKM <- RPKM[!grepl("STRG", RPKM$gene_id),] 

    write.csv(RPKM, file=paste0(DIR,"/analysis/07_quantification/",patients[i,1],"/",patients[i,1],"_table_of_counts_RPKM.csv"), row.names = FALSE, quote = FALSE)

    large_RPKM <- RPKM %>% pivot_longer(cols = !gene_id, names_to = "sample", values_to = "RPKM")

    df <- merge(large_RPKM, patients_full, by="sample", all=T)
    df$logRPKM <- log10(df$RPKM)
    df <- df %>% mutate(known_novel = case_when(grepl("STRG", gene_id) ~ "novel",
                                                        !grepl("STRG", gene_id) ~ "known"))
    colnames(featureLength) <- c("Length", "gene_id")
    df <- merge(df, featureLength, by="gene_id")

    ###length
    RPKM_length <- merge(RPKM, featureLength, by="gene_id")
    novel_genes <- df[df$known_novel == "novel",]
    known_genes <- df[df$known_novel == "known",]

    novel_genes_RPKM <- RPKM_length[grepl("STRG", RPKM_length$gene_id),] 
    known_genes_RPKM <- RPKM_length[!grepl("STRG", RPKM_length$gene_id),] 

    ###get those with length > 300
    novel_genes_300 <- novel_genes_RPKM[novel_genes_RPKM$Length >= 300,]
    dim(novel_genes_300)  

    filtered_RPKM <- rbind(known_genes_RPKM, novel_genes_300)

    filtered_df <- df %>% filter(df$gene_id %in% filtered_RPKM$gene_id)
    write.csv(filtered_RPKM, file=paste0(DIR,"/analysis/07_quantification/",patients[i,1],"/",patients[i,1],"_table_of_counts_300kb_RPKM.csv"), sep=",", row.names = FALSE, quote = FALSE)

    
    ###########PLOTS
    
    #ggplot(df, aes(x=Length, fill=known_novel)) +
    #geom_histogram(position = 'identity') +
    #  scale_fill_manual(values=c("#636363", "#fa9fb5")) +
    #scale_x_continuous(limits = c(0, 1000)) +
    #  ggtitle(paste0("Histogram of all gene lengths ",patients[i,1])) +
    #ylab("") +
    #xlab("Gene length")
    #ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_gene_length_histogram.png"))

    #ggplot(df, aes(x=Length, fill=known_novel)) +
    #geom_histogram(position = 'identity') +
    #  scale_fill_manual(values=c("#636363", "#fa9fb5")) +
    #scale_x_continuous(limits = c(0, 1000)) +
    #scale_y_continuous(limits = c(0, 3000)) +
    #  ggtitle(paste0("Histogram of all gene lengths ",patients[i,1]," (zoom)")) +
    #ylab("") +
    #xlab("Gene length")
    #ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_gene_length_histogram_zoom.png"))

    
    #ggplot(df, aes(x=logRPKM, color=known_novel)) +
    #    geom_density() +
    #      scale_color_manual("Known or Novel genes", values=c("#636363", "#fa9fb5")) +
    #    ggtitle(paste0("FPKM distribution ",patients[i,1])) +
    #    theme(plot.title = element_text(size=10, face="bold"))
    #ggsave(paste0(DIR, "/results/plots/",patients[i,1],"_FPKM_distribution.png"))

    #ggplot(filtered_df, aes(x=logRPKM, color=known_novel)) +
    #    geom_density() +
    #      scale_color_manual("Known or Novel genes", values=c("#636363", "#fa9fb5")) +
    #    ggtitle(paste0("FPKM distribution ",patients[i,1]," >300kb")) +
    #    theme(plot.title = element_text(size=10, face="bold"))
    #ggsave(paste0(DIR, "/results/plots/",patients[i,1],"_FPKM_distribution_300kb.png"))
}

Make summary file after quantification and after filtering by length

In [ ]:
%%bash -s "$DIR" "$patients_summary"

OUT=$1/results/quantification_num_genes_patient.txt
if [ -f "$OUT" ] ; then
    rm "$OUT"
fi
OUT2=$1/results/quantification_num_genes_patient_300kb.txt
if [ -f "$OUT2" ] ; then
    rm "$OUT2"
fi
sed 1d $2 | while IFS=, read patient normal tumor; do
echo -e ${patient}"\tTRANSCRIPTS\t"$(wc -l < $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_RPKM.csv) >> $OUT
echo -e ${patient}"\tANNOTATED TRANSCRIPTS\t"$(grep -v 'STRG' $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_RPKM.csv | wc -l) >> $OUT
echo -e ${patient}"\tNOVEL TRANSCRIPTS\t"$(grep 'STRG' $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_RPKM.csv | wc -l) >> $OUT

##300kb
echo -e ${patient}"\tTRANSCRIPTS\t"$(wc -l < $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_300kb_RPKM.csv) >> $OUT2
echo -e ${patient}"\tANNOTATED TRANSCRIPTS\t"$(grep -v 'STRG' $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_300kb_RPKM.csv | wc -l) >> $OUT2
echo -e ${patient}"\tNOVEL TRANSCRIPTS\t"$(grep 'STRG' $1/analysis/07_quantification/${patient}/${patient}_table_of_counts_300kb_RPKM.csv | wc -l)  >> $OUT2
done


## 08. Tumor specific

To filter what we consider as tumor specific, we apply a cutoff of FPKM lower or equal to 0,1 in normal samples and higher or equal than 1 in tumor samples

In [ ]:
if not os.path.exists(os.path.join(DIR,"analysis/08_tumor_specific")):
    os.mkdir(os.path.join(DIR,"analysis/08_tumor_specific"))

for i in patients.iloc[:,0]:
    INDIR=DIR+"/analysis/07_quantification/" + i
    OUTDIR=DIR+"/analysis/08_tumor_specific/" + i
    if not os.path.exists(OUTDIR):
        os.makedirs(OUTDIR)
        
    for file in os.listdir(INDIR):
        if file.endswith("_table_of_counts_300kb_RPKM.csv"):
            filepath = os.path.join(INDIR, file)
            outfile = OUTDIR + "/" + file.split("_")[0] + "_genes_N01_T1_300kb.csv"
            with open(filepath) as inp:
                with open(outfile, 'w') as out:
                    lines = inp.readlines()
                    out.write(lines[0])
                    del lines[0]
                    for line in lines:
                        line = line.rstrip()
                        line = line.split(",")
                        line[1] = float(line[1])
                        line[2] = float(line[2])
                        if line[1] <= 0.1 and line[2] >= 1: # expression cut - off established. FPKM <= 0.1 in normal and >= 1 in tumor
                            listToStr = ','.join(map(str, line))
                            out.write(listToStr + "\n")

Make summary file with tumor specific genes

In [ ]:
%%bash -s "$DIR" "$patients_summary"

OUT=$1/results/tumor_specific_genes_300kb.txt
if [ -f "$OUT" ] ; then
    rm "$OUT"
fi
sed 1d $2 | while IFS=, read patient normal tumor; do
echo -e ${patient}" TRANSCRIPTS: "$(wc -l < $1/analysis/08_tumor_specific/${patient}/${patient}_genes_N01_T1_300kb.csv) >> $OUT
echo -e "\tANNOTATED TRANSCRIPTS: "$(grep -v 'STRG' $1/analysis/08_tumor_specific/${patient}/${patient}_genes_N01_T1_300kb.csv | wc -l) >> $OUT
echo -e "\tNOVEL TRANSCRIPTS: "$(grep 'STRG' $1/analysis/08_tumor_specific/${patient}/${patient}_genes_N01_T1_300kb.csv | wc -l) >> $OUT

done


In [ ]:
%%R -i DIR,patients

# generate different csv files with novel and annotated transcripts tumor specific
all <- data.frame(gene=character(),
                  RPKM = numeric(),
                  Length=numeric(),
                  stringsAsFactors = FALSE)
for (i in 1:nrow(patients)) {
    input <- read.table(paste0(DIR,"/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_genes_N01_T1_300kb.csv"), header=TRUE, sep=",")
  
    names(input)[2] <- "normal"
    input$normal <- NULL
    input <- input %>% mutate(known_novel = case_when(grepl("STRG", gene_id) ~ "novel",
                                                      !grepl("STRG", gene_id) ~ "known"))
    #write.csv(input, file=paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_tumor_specific_genes_01_1FPKM_300kb.csv"), quote=FALSE, row.names = FALSE, sep=",")

    #Select the novel ones, the patient and the sample
    novel_tumor_specific <- input[grep("novel", input$known_novel),]
    write.table(novel_tumor_specific, file=paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_novel_tumor_specific_genes_01_1FPKM_300kb.csv"), quote=FALSE, row.names = FALSE, sep=",")
    
    #Select the known ones, the patient and the sample
    known_tumor_specific <- input[grep("known", input$known_novel),]
    write.table(known_tumor_specific, file=paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_01_1FPKM_300kb.csv"),  quote=FALSE, row.names = FALSE, sep=",")
    
    names(input)[2] <- "RPKM"
    input$patient <- patients[i,1]
    input$logRPKM <- log10(input$RPKM)
    all <- rbind(input,all)
 
}

####################################PLOTS####################################
####BARPLOT known/novel tumor specific genes
ggplot(all, aes(patient, ..count.., fill=known_novel)) +
      geom_bar(aes(fill=known_novel), position="dodge") +
      geom_text(aes(label=..count..), stat="count", size = 2, position=position_dodge(0.9)) +
      scale_fill_manual("Known or Novel genes", values=c("#636363", "#fa9fb5")) +
      ggtitle(paste0("Known/Novel tumor specific genes | FPKM 1 | 300kb")) +
      xlab("") +
      theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))
ggsave(paste0(DIR,"/results/plots/tumor_specific_genes_knownnovel_01_FPKM1_300kb.png"))

####BARPLOT known/novel proportion tumor specific genes
ggplot(all, aes(patient, ..count.., fill=known_novel)) +
      geom_bar(position="fill") +
      scale_fill_manual("Known or Novel genes", values=c("#636363", "#fa9fb5")) +
      ggtitle(paste0("Known/Novel tumor specific genes | FPKM 1 | 300kb")) +
      xlab("") +
      theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))
ggsave(paste0(DIR,"/results/plots/stacked_tumor_specific_genes_knownnovel_01_FPKM1_300kb.png"))

####HISTOGRAM gene length tumor speicifc genes facet by patient
ggplot(all, aes(x=Length, fill=known_novel)) +
      geom_histogram() +
      scale_fill_manual("Known or Novel genes", values=c("#636363", "#fa9fb5")) +
      scale_x_continuous(limits = c(0, 1000)) +
      ggtitle(paste0("Tumor specific gene length | FPKM 1 | 300kb")) +
      ylab("") +
      theme(axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10)) +
  facet_wrap(~patient, scales="free")
ggsave(paste0(DIR,"/results/plots/tumor_specific_gene_length_01_FPKM1_300kb.png"))

## Generate mini gtf tumor specific novel

In [ ]:
#for each csv file containing tumor specific genes, either novel or annotated, we generate a gtf file. 
#3 in total: a tumor specific one, another one only with novel features and another one with annotated features
counter=0
for p in patients_id:
    REF=DIR+"/analysis/06_stringtie/stringtie_reference_annotations/"+tumor_id[counter]+"_reference_annotation_sorted.gtf"
    with open(REF) as annotations:
        annotation_lines = annotations.readlines()
        CSV=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_genes_N01_T1_300kb.csv"
        g_id = []
        csv_file=pd.read_csv(CSV)
        g_id = list(csv_file.iloc[:,0])
        OUT=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_tumor_specific_genes_1FPKM_300kb.gtf"
        OUT_known=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_known_tumor_specific_genes_1FPKM_300kb.gtf"
        OUT_novel=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_novel_tumor_specific_genes_1FPKM_300kb.gtf"


        with open(OUT, 'w') as out:
            with open(OUT_novel, 'w') as novel:
                with open(OUT_known, 'w') as known:

                    for line in annotation_lines:
                        if line.startswith('#'):
                            next
                        else:
                            splitted_line = line.split(";")
                            ENSG = splitted_line[0]
                            ENSG = re.findall('"([^"]*)"', ENSG)
                            if ENSG[0] in g_id:
                                out.write(line)
                                if ENSG[0].startswith("ENSG"):
                                    known.write(line)
                                else:
                                    novel.write(line)
    counter = counter + 1

 

**Known**

Coding or non-coding?

For the annotated features, we are differentially interested in the coding and non-coding ones.

In [ ]:
%%R -i DIR,patients

colnames(patients) <- c("patient", "normal", "tumor")
patients_full <- patients %>% pivot_longer(cols = !patient, names_to = "normal_tumor", values_to = "sample")

full <- data.frame(gene_id = character(),
                      gene_type = factor(),
                      RPKM=numeric(),
                   patient=factor(),
                      stringsAsFactors = FALSE)

for (i in 1:nrow(patients)) {
  
  file_CDS <- paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_1FPKM_300kb.gtf")
  csv <- read.csv(paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_01_1FPKM_300kb.csv"))
  names(csv)[2] <- "RPKM"
  names(csv)[1] <- "gene_id"
  
  gtf <- as.data.frame(rtracklayer::import(file_CDS)) # tumor specific annotated gtf file
  NOCDS <- gtf[gtf$gene_type != "protein_coding",]
  export(NOCDS, paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_1FPKM_300kb_NOCDS.gtf"))
  
  CDS <- gtf[gtf$gene_type == "protein_coding",]
  export(CDS, paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_1FPKM_300kb_CDS.gtf"))
  
  ##NOCDS
  clean_gtf <- gtf[gtf$type == "gene",]
  clean_gtf <- clean_gtf %>% select(seqnames, type,gene_id, transcript_id, gene_type, transcript_type, gene_name, transcript_name)
  
  df <- merge(clean_gtf, csv, by="gene_id")
  df$logRPKM <- log10(df$RPKM)
  df <- df %>% mutate(coding = case_when(grepl("protein_coding", gene_type) ~ "coding",
                                                    !grepl("protein_coding", gene_type) ~ "non-coding"))
  df$patient <- patients[i,1]
  full <- rbind(full, df)
  

  clean_gtf_NOCDS <- NOCDS[NOCDS$type == "gene",]
  clean_gtf_NOCDS <- clean_gtf_NOCDS %>% select(seqnames, type,gene_id, gene_type, gene_name)
  
  df_NOCDS <- merge(clean_gtf_NOCDS, csv, by="gene_id")
  df_NOCDS$logRPKM <- log10(df_NOCDS$RPKM)
  
  selected_NOCDS <- df_NOCDS[grep("lncRNA|^processed_pseudogene", df_NOCDS$gene_type),] #we are only interested in long non-coding RNAs and processed pseudogenes
  
  write.csv(selected_NOCDS, file=paste0(DIR,"/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.csv"), row.names = FALSE, quote = FALSE)

    
###################################PLOTS PER PATIENT#####################################33
#  ggplot(df, aes(x=logRPKM, color=coding)) +
#    geom_density() +
#    scale_color_manual("Coding or non-coding genes", values=c("#636363", "#7fcdbb")) +
#    ggtitle(paste0("FPKM distribution ",patients[i,1])) +
#    theme(plot.title = element_text(size=10, face="bold"))
#  ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_coding_tumor_specific_genes_logdistribution_FPKM1_300kb.png"))


#  ggplot(df, aes(x=coding, fill=coding)) +
#    geom_bar() +
#    geom_text(aes(label=..count..), stat="count", size = 2, position=position_dodge(0.9)) +
#    scale_fill_manual("Coding or non-coding genes", values=c("#636363", "#7fcdbb")) +
#    ggtitle(paste0("Coding or non-coding tumor specific genes ",patients[i,1], " | FPKM 1 | 300kb")) +
#    xlab("") +
#    theme(axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))
#  ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_coding_tumor_specific_genes_FPKM1_300kb.png"))
  
   
#   ggplot(selected_NOCDS, aes(x=logRPKM, color=gene_type)) +
#    geom_density() +
#     scale_color_manual("Gene type", values=c("#addd8e", "#a6bddb")) +
#     ggtitle(paste0("FPKM distribution ",patients[i,1]," non-coding genes")) +
#    theme(plot.title = element_text(size=10, face="bold"))
#  ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_non-coding_selected_tumor_specific_genes_logdistribution_FPKM1_300kb.png"))
  
  
#  ggplot(selected_NOCDS, aes(x=gene_type, fill=gene_type)) +
#    geom_bar() +
#    geom_text(aes(label=..count..), stat="count", size = 2, position=position_dodge(0.9)) +
#    scale_fill_manual("Gene type", values=c("#addd8e", "#a6bddb")) +
#    ggtitle(paste0("Non-coding tumor specific genes ",patients[i,1], " | FPKM 1 | 300kb")) +
#    xlab("") +
#    theme(axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))
#  ggsave(paste0(DIR,"/results/plots/",patients[i,1],"_non-coding_selected_tumor_specific_genes_FPKM1_300kb.png"))
  
}
###################################PLOTS#####################################33

ggplot(full, aes(x=coding, fill=coding)) +
  geom_bar() +
  geom_text(aes(label=..count..), stat="count", size = 2, position=position_dodge(0.9)) +
  scale_fill_manual("Coding or non-coding genes", values=c("#636363", "#7fcdbb")) +
  ggtitle(paste0("Coding or non-coding tumor specific genes | FPKM 1 | 300kb")) +
  xlab("") +
  theme(axis.text.x = element_blank(), axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))+
  facet_wrap(~patient, scales = "free")
ggsave(paste0(DIR,"/results/plots/coding_tumor_specific_genes_FPKM1_300kb.png"))

ggplot(full, aes(x=patient, ..count.., fill=coding)) +
  geom_bar(position="fill") +
  scale_fill_manual("Coding or non-coding genes", values=c("#636363", "#7fcdbb")) +
  ggtitle(paste0("Coding or non-coding tumor specific genes | FPKM 1 | 300kb")) +
  xlab("") +
  theme(axis.text.x = element_text(angle = 45, hjust=1), axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 10))
ggsave(paste0(DIR,"/results/plots/coding_tumor_specific_genes_FPKM1_300kb_stacked.png"))

ggplot(full, aes(x=logRPKM, color=coding)) +
  geom_density() +
  scale_color_manual("Coding or non-coding genes", values=c("#636363", "#7fcdbb")) +
  ggtitle("FPKM distribution coding/non-coding genes") +
  theme(plot.title = element_text(size=10, face="bold")) +
  facet_wrap(~patient, scales="free")
ggsave(paste0(DIR,"/results/plots/coding_tumor_specific_genes_logdistribution_FPKM1_300kb.png"))


Generate minigtf of NOCDS selected (lncRNA and processed_pseudogenes)

In [ ]:
counter=0
for p in patients_id:
    print(p," is being read")
    REF=DIR+"/analysis/06_stringtie/stringtie_reference_annotations/"+tumor_id[counter]+"_reference_annotation_sorted.gtf"
    with open(REF) as annotations:
        annotation_lines = annotations.readlines()
        CSV=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.csv"
        csv_file=pd.read_csv(CSV)
        g_id = list(csv_file.iloc[:,0])
        OUT_known=DIR+"/analysis/08_tumor_specific/"+p+"/" + p + "_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.gtf"

        with open(OUT_known, 'w') as out:
            for line in annotation_lines:
                if line.startswith('#'):
                    next
                else:
                    splitted_line = line.split(";")
                    ENSG = splitted_line[0]
                    ENSG = re.findall('"([^"]*)"', ENSG)
                    if ENSG[0] in g_id:
                        out.write(line)

    counter = counter + 1

 

In [ ]:
#more than one transcript line per gene_id 
#these are the unique ones
OUT=DIR+"/results/transcripts_genes_tumor_specific.txt"
with open(OUT,'w') as out:
    for p in patients_id:
        file=DIR+"/analysis/08_tumor_specific/"+p+"/"+p+"_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.gtf"
        genes_list = []
        with open(file) as inp:
            lines=inp.readlines()
            for line in lines:
                if "\ttranscript\t" in line:
                    splitted_line = line.split(";")
                    gene_id = splitted_line[0]
                    ENSG = re.findall('"([^"]*)"', gene_id)
                    genes_list.append(ENSG[0])
        genes_set = set(genes_list)
        out.write(p+"gene"+str(len(genes_set)))
        out.write(p+"transcript"+str(len(genes_list)))

Get fasta of noncoding selected

In [ ]:
%%bash -s "$DIR" "$patients_summary" "$GENOMEDIR"

export PATH=/genomics/users/marta/tools/gffread-0.12.7.Linux_x86_64/:$PATH
GENOME_FASTA=$3/GRCh38/GRCh38.primary_assembly.genome.fa

while IFS=, read p normal tumor; do
    file=$1/analysis/08_tumor_specific/${p}/${p}_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.gtf
    
    #get fasta
    gffread --attrs gene_name,transcript_type,transcript_name -w ${file%%.*}.fa -g $GENOME_FASTA $file

    #count fasta sequences to confirm the process
    echo -e $p"\t"$(grep '>' ${file%%.*}.fa | wc -l)
    
    #replace spaces by ;
    sed -i 's/\ /;/g' ${file%%.*}.fa
done < $2


In [ ]:
#we will work at gene level. For genes with more than one transcript, we keep the longest one
for p in patients_id:
    g_dict= dict()
    full = dict()
    file=DIR+"/analysis/08_tumor_specific/"+p+"/"+p+"_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected.fa"
    OUT=DIR+"/analysis/08_tumor_specific/"+p+"/"+p+"_known_tumor_specific_genes_1FPKM_300kb_NOCDS_selected_gene.fa"
    with open(OUT, 'w') as out:
        for seq_record in SeqIO.parse(file, 'fasta'):  # (generator)   
            gene_name = seq_record.id.split(";")[1]
            gene_name = gene_name.split("=")[1]
            if gene_name in g_dict.keys():
                if len(seq_record.seq) > len(g_dict[gene_name]): # get the longest transcript per gene
                    g_dict[gene_name] = seq_record.seq
                    full[gene_name] = seq_record.id
                else:       
                    next
            else:
                g_dict[gene_name] = seq_record.seq
                full[gene_name] = seq_record.id
                
        print(len(full))
        for identif, seq in g_dict.items():
            out.write(">%s\n%s\n" %(full[identif], seq))


**CDS**

In [ ]:
%%bash -s "$DIR" "$patients_summary" "$GENOMEDIR"

export PATH=/genomics/users/marta/tools/gffread-0.12.7.Linux_x86_64/:$PATH
GENOME_FASTA=$3/GRCh38/GRCh38.primary_assembly.genome.fa

cat $2 | while IFS=, read p normal tumor; do
    file=$1/analysis/08_tumor_specific/${p}/${p}_known_tumor_specific_genes_1FPKM_300kb_CDS.gtf
    
    #get fasta
    gffread --attrs gene_name,transcript_type,transcript_name -x ${file%%.*}.fa -g $GENOME_FASTA $file
    
    #count fasta sequences to confirm the process
    echo -e $p"\t"$(grep '>' ${file%%.*}.fa | wc -l)
    
    #replace spaces by ;
    sed -i 's/\ /;/g' ${file%%.*}.fa
done

In [ ]:
#we will work at gene level. For genes with more than one transcript, we keep the longest one
for p in patients_id:
    g_dict= dict()
    full = dict()
    file=DIR+"/analysis/08_tumor_specific/"+p+"/"+p+"_known_tumor_specific_genes_1FPKM_300kb_CDS.fa"
    OUT=DIR+"/analysis/08_tumor_specific/"+p+"/"+p+"_known_tumor_specific_genes_1FPKM_300kb_CDS_gene.fa"
    with open(OUT, 'w') as out:
        for seq_record in SeqIO.parse(file, 'fasta'):  # (generator)   
            gene_name = seq_record.id.split(";")[1]
            gene_name = gene_name.split("=")[1]
            if gene_name in g_dict.keys():
                if len(seq_record.seq) > len(g_dict[gene_name]): # get longest transcript per gene
                    g_dict[gene_name] = seq_record.seq
                    full[gene_name] = seq_record.id
                else:
                    next
            else:
                g_dict[gene_name] = seq_record.seq
                full[gene_name] = seq_record.id
                
        print(len(full))
        for identif, seq in g_dict.items():
            out.write(">%s\n%s\n" %(full[identif], seq))


For protein-coding genes, how many of them are shared between patients ?

In [ ]:
%%bash -s "$DIR"

mkdir $1/analysis/09_CIPHER

In [ ]:
%%R -i DIR,patients

colnames(patients) <- c("patient", "normal", "tumor")
patients_full <- patients %>% pivot_longer(cols = !patient, names_to = "normal_tumor", values_to = "sample")

all <- data.frame(transcript_id = character(),
                  transcript_type = factor(), 
                  transcript_name = character(),
                  patient = factor(),
                  stringsAsFactors=FALSE)

for (i in 1:nrow(patients)) {
  
  df <- as.data.frame(rtracklayer::import(paste0(DIR, "/analysis/08_tumor_specific/",patients[i,1],"/",patients[i,1],"_known_tumor_specific_genes_1FPKM_300kb_CDS.gtf")))
  df <- df[df$type == "transcript",]
  df <- df[df$transcript_type == "protein_coding",] #only the protein coding ones at transcript level
  df$patient <- patients[i,1]
  df_selected <- df %>% select(transcript_id, gene_id, gene_name)
  
  all <- rbind(all,df_selected)
}  

times <- all %>% count(transcript_id)
times_ordered <- times[order(times$n, decreasing=TRUE),]
print(times_ordered %>% head)

total <- merge(times_ordered,all, by="transcript_id")
total <- total[,c(1,3,4,2)]
total <- total[order(total$n, decreasing=TRUE),]
total_unique <- total %>% distinct()
total_unique$transcript_id <- sub("*\\.[0-9]", "", total_unique$transcript_id)
total_unique$gene_id <- sub("*\\.[0-9]", "", total_unique$gene_id)
total_unique %>% head

write.csv(total_unique, file.path(DIR,"analysis/09_CIPHER/common_coding_genes.csv"),row.names = FALSE, quote = FALSE)

ggplot(times, aes(x=n)) +
  geom_bar(fill="#636363") +
  geom_text(stat="count", aes(label=..count..), vjust=-1, size = 2.5) +
  ggtitle("Common non-coding genes across patients | lncRNA & processed_pseudogenes") +
  theme(axis.title.x = element_blank(), axis.title.y = element_blank(), plot.title = element_text(face="bold", size = 9), legend.position = "none") 
ggsave(file.path(DIR,"results/plots/common_coding_genes_patients.png"))


### GFFCOMPARE

Create only one newly assembled tumor-specific transcriptome

In [ ]:
#first modify the idenifiers so we can get information of the patient they belong only looking to the identifier
try:
    os.makedirs(DIR+"/analysis/gffcompare")
except:
    print("gffcompare directory already exists for dataset: %s" %(PROJECT))
    
for p in patients_id:
    file=DIR+"/analysis/08_tumor_specific/"+str(p)+"/"+str(p)+"_novel_tumor_specific_genes_1FPKM_300kb.gtf"
    with open (file) as inp:
        outfile=DIR+"/analysis/gffcompare/"+str(p)+"_novel_tumor_specific_genes_1FPKM_300kb_MODIF.gtf"
        with open(outfile, 'w') as out:
            lines=inp.readlines()
            for line in lines:
                final_line = line.replace('STRG', 'STRG.'+str(p))
                out.write(final_line)
            

In [ ]:
%%bash -s "$DIR" "$patients_summary"

declare -a listfiles

cat $2 | while read patient normal tumor; do
    listfiles=(${listfiles[@]} $1/analysis/08_tumor_specific/$patient/${patient}_novel_tumor_specific_genes_1FPKM_300kb.gtf) # Add new element at the end of the array
done 

cd $1/analysis/gffcompare

printf "%s\n" "${listfiles[@]}" > list_novel_tumorsp_files.txt

module load gffcompare/20190227

gffcompare -i list_novel_tumorsp_files.txt 


## 12. Hepatocarcinoma Biomarkers

**Gene expression downregulation**

- MT1M/ENSG00000205364.4 (Mao et al., 2012, Carcinogenesis 33 : 2568–2577). 

- CDKN2A/ENSG00000147889.18 (The Cancer Genome Atlas Research Network, 2017, Cell 169, 1327–1341. Comprehensive and Integrative Genomic Characterization of Hepatocellular Carcinoma).


**Gene expression upregulation**

- TERT/ENSG00000164362.21 (The Cancer Genome Atlas Research Network, 2017, Cell 169, 1327–1341. Comprehensive and Integrative Genomic Characterization of Hepatocellular Carcinoma).

- GABRD/ENSG00000187730.9, THBDS4/ENSG00000113296.14 (!THBS4) (Sun et al., 2018. An integrated analysis of genome-wide DNA methylation and gene expression data in hepatocellular carcinoma. FEBS Open Bio).

- LINC01419/ENSG00000253898.1, HAGLR/ENSG00000224189.8, LINC00176/ENSG00000196421.8 (Unfried et al., 2019. Identification of Coding and Long Noncoding RNAs Differentially Expressed in Tumors and Preferentially Expressed in Healthy Tissues. Cancer Res).


In [ ]:
biomarkers=pd.read_csv(os.path.join(GENOMEDIR,"hepatocarcinoma_biomarkers.txt"))
try:
    os.mkdir(DIR+"/analysis/12_biomarkers")
except:
    print("12_biomarkers directory already exists, some data may be replaced")

for p in patients_id:
    print(p)
    present = pd.DataFrame()
    merged_df = pd.DataFrame()
    full_df = pd.DataFrame()
    INDIR=DIR+"/analysis/08_tumor_specific/"+p+"/"
    file=p+"_known_tumor_specific_genes_01_1FPKM_300kb.csv"
    df=pd.read_csv(os.path.join(INDIR,file), names=['gene_id','FPKM','Length','known_novel']) 
   
    out=DIR+"/analysis/12_biomarkers/"+p+"_hepato_biomarkers.csv" 
    for each in biomarkers['gene_id']:
        if df['gene_id'].str.contains(each).any():
            present = present.append(df[df['gene_id'] == each], ignore_index=True)
            merged_df = pd.merge(present,biomarkers.loc[biomarkers['gene_id'] == each], on = "gene_id")
            full_df=full_df.append(merged_df, ignore_index=False)
    full_df.to_csv(out, index=False, header=True)

In [ ]:
%%R -i GENOMEDIR

colnames(patients) <- c("patient", "normal", "tumor")
patients_full <- patients %>% pivot_longer(cols = !patient, names_to = "normal_tumor", values_to = "sample")

biomarkers <- read.csv(paste0(GENOMEDIR,"/hepatocarcinoma_biomarkers.txt"))

full <- data.frame(gene_id = character(),
                      sample = factor(),
                      Length = numeric(),
                      FPKM = numeric(),
                   normal_tumor = factor(),
                      patient=factor(),
                   gene_name = character(),
                   type=factor(),
                      stringsAsFactors = FALSE)

for (i in 1:nrow(patients)) {
  csv <- read.csv(paste0(DIR,"/analysis/07_quantification/",patients[i,1],"/",patients[i,1],"_table_of_counts_300kb_RPKM.csv"))
  names(csv)[1] <- "gene_id"
  csv_long <- csv %>% pivot_longer(cols = starts_with('SRR'), names_to = "sample", values_to = "FPKM")
  csv_long <- merge(csv_long, patients_full, by="sample")
  biomarkers_in_sample <- csv_long[csv_long$gene_id %in% biomarkers$gene_id,]
  biomarkers_in_sample <- merge(biomarkers_in_sample, biomarkers, by="gene_id")
  biomarkers_in_sample$logFPKM <- log10(biomarkers_in_sample$FPKM)

  full <- rbind(full,biomarkers_in_sample)
}

write.csv(full, paste0(DIR,"/analysis/12_biomarkers/total_biomarkers_values.csv"), row.names=FALSE, quote=FALSE)
ggplot(full, aes(x=gene_name, y=logFPKM)) +
geom_boxplot(aes(fill=normal_tumor)) +
geom_point(position=position_jitterdodge(jitter.width=1, dodge.width = 0), 
         pch=21, aes(fill=normal_tumor), show.legend = F, size=1) +
scale_fill_manual("Adjacent or Tumor sample", values=c("#31a354", "#de2d26")) +
ggtitle("Gene expression cancer biomarkers") +
  theme(axis.text.x = element_text(angle = 45, hjust=1), plot.title = element_text(face="bold", size = 10))
ggsave(paste0(DIR,"/results/plots/cancer_markers_expression_all.png"), height=12, width=8)
